In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
df = pd.read_csv('data1.csv')
df = df.dropna()
df

,Date,NG,FGT,HZT,Stack,FeedRate,RPM,Cooler,CO,SSA,BA,ESPIT,ESPOT,ESPIP,PAP
0,29/05/24 5:00,800.93,421.06,1156.61,176.30,8.19,40.00,33.49,0.004,5.10,2.0,326.18,101.47,128.56,761.24
1,29/05/24 6:00,800.76,417.51,1156.87,179.29,8.20,40.00,33.89,0.004,5.80,2.0,326.59,100.70,131.05,760.15
2,29/05/24 7:00,799.87,413.21,1147.42,179.27,8.20,40.00,34.55,0.004,5.80,6.0,330.47,101.01,130.95,759.27
3,29/05/24 8:00,818.03,410.47,1143.88,179.95,8.36,41.03,35.38,0.004,6.40,2.0,329.80,102.44,131.60,757.93
4,29/05/24 9:00,824.31,404.56,1143.06,180.51,8.50,42.00,36.75,0.004,5.40,7.0,322.15,100.63,132.00,760.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,9/16/24 08:00,818.63,323.05,1273.90,180.00,8.90,46.00,43.79,0.009,1.20,0.0,161.26,135.48,121.67,700.36
1522,9/16/24 09:00,819.15,321.83,1274.13,178.75,8.90,46.00,44.51,0.009,1.20,0.0,159.96,135.53,121.95,698.40
1523,9/16/24 10:00,819.66,320.81,1283.55,179.97,8.96,46.01,44.97,0.009,1.19,0.0,159.46,135.70,123.10,697.09
1524,9/16/24 11:00,819.77,316.08,1285.53,183.23,9.16,46.00,48.23,0.015,1.16,0.0,155.37,134.46,126.15,695.44


from matplotlib import pyplot as plt
df['NG'].plot(kind='hist', bins=20, title='NG')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['FGT'].plot(kind='hist', bins=20, title='FGT')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['HZT'].plot(kind='hist', bins=20, title='HZT')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Stack'].plot(kind='hist', bins=20, title='Stack')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='FGT', y='HZT', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='HZT', y='Stack', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='Stack', y='FeedRate', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Date']
  ys = series['NG']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('NG')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Date']
  ys = series['FGT']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Date')
_ = plt.ylabel('FGT')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1514 entries, 0 to 1525
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1514 non-null   object 
 1   NG        1514 non-null   float64
 2   FGT       1514 non-null   float64
 3   HZT       1514 non-null   float64
 4   Stack     1514 non-null   float64
 5   FeedRate  1514 non-null   float64
 6   RPM       1514 non-null   float64
 7   Cooler    1514 non-null   float64
 8   CO        1514 non-null   float64
 9   SSA       1514 non-null   float64
 10  BA        1514 non-null   float64
 11  ESPIT     1514 non-null   float64
 12  ESPOT     1514 non-null   float64
 13  ESPIP     1514 non-null   float64
 14  PAP       1514 non-null   float64
dtypes: float64(14), object(1)
memory usage: 221.5+ KB


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd  # Assuming you are using pandas for data manipulation

# Assuming your data is stored in a pandas DataFrame named 'df'
# Replace 'df' with your actual DataFrame name if it's different

# Define your features (X) and target variable (y)
X = df.drop(columns=['Date'])  # Features are all columns except 'Molochite'
y = df['SSA']  # Target variable is 'Molochite'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=0.7,  # 30% of the data will be used for testing
    random_state=0  # Set random_state for reproducibility
)

# Now you can use X_train, X_test, y_train, y_test in your machine learning model


In [ ]:
X_train.head()

,NG,FGT,HZT,Stack,FeedRate,RPM,Cooler,CO,SSA,BA,ESPIT,ESPOT,ESPIP,PAP
546,824.69,361.72,1287.94,160.07,8.69,42.00,38.20,0.005,1.50,0.1,154.56,122.36,84.25,719.48
249,850.79,364.11,1284.12,177.86,9.10,47.45,34.68,0.005,1.09,2.0,160.80,99.91,96.49,680.57
652,849.93,418.82,1290.76,170.18,8.70,42.00,38.22,0.006,1.30,0.1,167.18,131.63,107.65,710.28
1056,801.13,309.30,1326.12,204.02,9.14,47.00,47.01,0.008,1.33,0.0,155.38,125.48,144.39,737.06
655,850.07,427.22,1296.16,170.05,8.70,42.00,35.40,0.006,1.38,0.1,164.30,128.95,93.74,711.88


In [ ]:
#model creation
model = RandomForestRegressor()

In [ ]:
#model training
y = df['SSA']
y = y[:int(len(df)-1)]
x = df[['NG','FGT','HZT','Stack','FeedRate','CO','Cooler','RPM','BA','ESPIT','ESPOT','ESPIP','PAP' ]]
x = x[:int(len(df)-1)]
model.fit(x,y)

RandomForestRegressor()

In [ ]:
#testing
predictions = model.predict(x)
print('The model score is:', model.score(x,y))

The model score is: 0.9584677217504053


In [ ]:
# Allow user input for each parameter
try:
    NG = float(input("Enter NG value: "))
    FGT = float(input("Enter FGT value: "))
    HZT = float(input("Enter HZT value: "))
    Stack = float(input("Enter Stack value: "))
    FeedRate = float(input("Enter Feed Rate value: "))
    CO = float(input("Enter CO value: "))
    Cooler = float(input("Enter Cooler value: "))
    RPM = float(input("Enter RPM value: "))
    BA = float(input("Enter BA value: "))
    ESPIP = float(input("Enter ESPIP value: "))
    ESPOT = float(input("Enter ESPOT value: "))
    ESPIT = float(input("Enter ESPIT value: "))
    PAP = float(input("Enter PAP value: "))
except ValueError:
    print("Invalid input. Please enter numeric values.")
    exit()

# Prepare input data for prediction in DataFrame format
new_data = pd.DataFrame({
    'NG': [NG], 'FGT': [FGT], 'HZT': [HZT], 'Stack': [Stack],
    'FeedRate': [FeedRate], 'CO': [CO], 'Cooler': [Cooler],
    'RPM': [RPM], 'BA': [BA], 'ESPIT': [ESPIT],
    'ESPOT': [ESPOT], 'ESPIP': [ESPIP], 'PAP': [PAP]
})

# Make prediction
try:
    prediction = model.predict(new_data)
    print('The model predicts the SSA analysis to be:', prediction[0])
except Exception as e:
    print(f"An error occurred during prediction: {e}")

Enter NG value: 888
Enter FGT value: 882
Enter HZT value: 8128
Enter Stack value: 89798
Enter Feed Rate value: 1289
Enter CO value: 123
Enter Cooler value: 12312
Enter RPM value: 123
Enter BA value: 234
Enter ESPIP value: 123
Enter ESPOT value: 448
Enter ESPIT value: 234
Enter PAP value: 12
The model predicts the SSA analysis to be: 9.806499999999996


In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('model.pkl','wb'))